In [1]:
import json
import yaml
import pandas as pd
from tqdm import tqdm
import re
import numpy as np

pd.set_option('future.no_silent_downcasting', True)

# DATA

## Load data and tokens

In [2]:
with open('data/entities.json', 'r') as f:
    data = json.load(f)

In [3]:
with open('data/tokens.yml', 'r') as f:
    tokens = yaml.safe_load(f)
    
token_to_col = {}
for col in tokens.keys():
    token = tokens[col]['start']
    token_to_col[token] = col

## Preprocessing

In [4]:
def blank_dict(tokens=tokens):
    d = {}
    for col in list(tokens.keys()):
        d[col] = None
    return d

def split_by_token(line, token_to_col=token_to_col):
    tokens_to_split = list(token_to_col.keys())
    pattern = '(' + '|'.join(re.escape(token) for token in tokens_to_split) + ')'
    
    splits = re.split(pattern, line)
    # Filter out empty strings and trim kept strings
    splits = [part.strip() for part in splits if part]
    return splits

def split_to_dict(split, token_to_col=token_to_col, dict_split=None):
    if dict_split is None:
        dict_split = blank_dict()
    for i in range(0, len(split), 2):
        token = split[i]
        element = split[i+1]
        column = token_to_col[token]
        dict_split[column] = element
    return dict_split

In [5]:
df_dict = {}
counter = 0
for key in tqdm(data.keys()): 
    for line in data[key].split('\n'):
        try:
            split = split_by_token(line)
            split_dict = split_to_dict(split)
            df_dict[counter] = split_dict
                    
            counter += 1
        except:
            pass

100%|██████████| 1218/1218 [00:00<00:00, 3143.14it/s]


In [6]:
df = pd.DataFrame().from_dict(df_dict, orient='index').fillna(value=np.nan)

indices_to_remove = []
for i in range(len(df)):
    if np.all(df.iloc[i].isna()):
        indices_to_remove.append(i)

df = df.loc[~df.index.isin(indices_to_remove)]

In [7]:
X = df.drop(columns=['surname_household'])
y = df['surname_household'].apply(
                                lambda x: 0 if pd.isna(x) else 1
                            )

In [8]:
X

,age,birth_date,civil_status,education_level,employer,firstname,link,lob,maiden_name,nationality,observation,occupation,surname
0,25,NaN,Garçon,NaN,NaN,Cyrille,NaN,NaN,NaN,française,NaN,menuisier,Breton
1,30,NaN,Garçon,NaN,NaN,Auguste,NaN,NaN,NaN,Piémontaise,NaN,vitrier,NaN
2,24,NaN,Garçon,NaN,NaN,Pierre,NaN,NaN,NaN,Piémontaise,NaN,vitrier,NaN
3,48,NaN,Homme marié,NaN,NaN,Alexandre,NaN,NaN,NaN,française,NaN,prop re,NaN
4,30,NaN,NaN,NaN,NaN,Zélie,sa fe,NaN,NaN,française,NaN,prop re,Vignat
...,...,...,...,...,...,...,...,...,...,...,...,...,...
25433,NaN,1869,NaN,NaN,NaN,Marie,chef,Pailharès,NaN,idem,NaN,NaN,NaN
25434,NaN,1863,NaN,NaN,Cara,Marie,chef,St Naz en Royans,NaN,idem,NaN,ouv chaus res,NaN
25435,NaN,1886,NaN,NaN,Baretto,Nello,chef,Castel,NaN,italienne,NaN,manoeuvre,NaN
25436,NaN,1887,NaN,NaN,NaN,Annunziata,épouse,idem,NaN,idem,NaN,NaN,Berni-Laureti


# MODELLING

## Gradient Boosing

In [9]:
from sklearn.preprocessing import OrdinalEncoder
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.ensemble import HistGradientBoostingClassifier
from sklearn.metrics import log_loss

### Ordinal Encoding

In [10]:
enc = OrdinalEncoder()
X_encoded = enc.fit_transform(X)

In [11]:
X_train_enc, X_test_enc, y_train_enc, y_test_enc = train_test_split(X_encoded, y, test_size=0.20, stratify=y)

### All information

In [ ]:
param_grid = {
    "learning_rate": [1e-1, 1e-2, 1e-3],
    "max_leaf_nodes": [15, 31, 100],
    "max_depth": [None, 5, 10, 20],
    "min_samples_leaf": [10, 20, 50],
    "l2_regularization": [0.0, 0.1, 1.0]
}

model = HistGradientBoostingClassifier(max_iter=10_000, early_stopping=True, class_weight='balanced', validation_fraction=0.2)
grid_search = GridSearchCV(model, param_grid, cv=3, scoring="accuracy", n_jobs=-1, verbose=0)

grid_search.fit(X_train_enc, y_train_enc)

In [13]:
# Print the best parameters found
print("Best parameters:", grid_search.best_params_)

Best parameters: {'l2_regularization': 0.1, 'learning_rate': 0.1, 'max_depth': None, 'max_leaf_nodes': 31, 'min_samples_leaf': 10}


In [14]:
# Get the best model from the grid search
clf = grid_search.best_estimator_

In [15]:
print(f"Train accuracy: {clf.score(X_train_enc, y_train_enc):.3f}")
print(f"Test accuracy: {clf.score(X_test_enc, y_test_enc):.3f}")

Train accuracy: 0.999
Test accuracy: 0.997


In [16]:
print(f'Train CE loss = {log_loss(y_train_enc, clf.predict_proba(X_train_enc)):.4f}')
print(f'Test CE loss = {log_loss(y_test_enc, clf.predict_proba(X_test_enc)):.4f}')

Train CE loss = 0.0036
Test CE loss = 0.0101


### Remove surname

There seems to be a direct inverse correlation between the presence of the surname and being a household leader, in the sense that a household leader has a household surname given but has no surname, and vice versa for non household leaders. Hence, it might be to easy for the model as it only has to check whether there is a surname or not to predict the truth. <br>
Hence, we remove this column of information from the training data to see if the model is still able to have great results with the remaining information.

In [17]:
enc = OrdinalEncoder()
X_encoded_no_surname = enc.fit_transform(X.drop(columns=['surname']))

In [18]:
X_train_enc_no_surname, X_test_enc_no_surname, y_train_enc_no_surname, y_test_enc_no_surname = train_test_split(X_encoded_no_surname, y, test_size=0.20, stratify=y)

In [ ]:
model = HistGradientBoostingClassifier(max_iter=10_000, early_stopping=True, class_weight='balanced', validation_fraction=0.2)
grid_search = GridSearchCV(model, param_grid, cv=3, scoring="accuracy", n_jobs=-1, verbose=0)

grid_search.fit(X_train_enc_no_surname, y_train_enc_no_surname)

In [20]:
# Print the best parameters found
print("Best parameters while not using surname:", grid_search.best_params_)

Best parameters while not using surname: {'l2_regularization': 0.0, 'learning_rate': 0.1, 'max_depth': None, 'max_leaf_nodes': 100, 'min_samples_leaf': 20}


In [38]:
# Get the best model from the grid search
clf_no_surname = grid_search.best_estimator_
clf_no_surname.fit(X_train_enc_no_surname, y_train_enc_no_surname);

In [39]:
print(f"Train accuracy no surname: {clf_no_surname.score(X_train_enc_no_surname, y_train_enc_no_surname):.3f}")
print(f"Test accuracy no surname: {clf_no_surname.score(X_test_enc_no_surname, y_test_enc_no_surname):.3f}")

Train accuracy no surname: 0.941
Test accuracy no surname: 0.922


In [40]:
print(f'Train CE loss no surname = {log_loss(y_train_enc_no_surname, clf_no_surname.predict_proba(X_train_enc_no_surname)):.4f}')
print(f'Test CE loss no surname = {log_loss(y_test_enc_no_surname, clf_no_surname.predict_proba(X_test_enc_no_surname)):.4f}')

Train CE loss no surname = 0.1544
Test CE loss no surname = 0.2182


## Fine-Tune pretrained model

In [65]:
X_str = np.array([' '.join([str(x) for x in X.iloc[i].dropna(inplace=False).values]) for i in range(len(X))], dtype=str)
X_str_train, X_str_test, y_str_train, y_str_test = train_test_split(X_str, y, test_size=0.25, stratify=y)
X_str_val, X_str_test, y_str_val, y_str_test = train_test_split(X_str_test, y_str_test, test_size=0.4, stratify=y_str_test)

In [73]:
import torch
from transformers import BertTokenizer, BertForSequenceClassification
from sklearn.model_selection import train_test_split
from torch.utils.data import DataLoader, TensorDataset
from transformers import AdamW
from tqdm import tqdm

# Step 1: Load pre-trained French language model and tokenizer
model_name = "dbmdz/bert-base-french-europeana-cased"
tokenizer = BertTokenizer.from_pretrained(model_name)
model = BertForSequenceClassification.from_pretrained(model_name, num_labels=2)  # Assuming binary classification

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at dbmdz/bert-base-french-europeana-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [74]:
# Step 2: Tokenize text data
# Tokenize input text
train_inputs = tokenizer(list(X_str_train), padding=True, truncation=True, return_tensors="pt")
val_inputs = tokenizer(list(X_str_val), padding=True, truncation=True, return_tensors="pt")
test_inputs = tokenizer(list(X_str_test), padding=True, truncation=True, return_tensors="pt")

# Convert labels to tensor
train_labels = torch.tensor(list(y_str_train))
val_labels = torch.tensor(list(y_str_val))
test_labels = torch.tensor(list(y_str_test))

In [75]:
# Step 4: Prepare DataLoader for training and validation sets
def to_loader(inputs, labels, batch_size=32, shuffle=True):
    dataset = TensorDataset(inputs["input_ids"], inputs["attention_mask"], labels)
    loader = DataLoader(dataset, batch_size=batch_size, shuffle=shuffle)
    return dataset, loader

train_dataset, train_loader = to_loader(train_inputs, train_labels, shuffle=True)
val_dataset, val_loader = to_loader(val_inputs, val_labels, shuffle=False)
test_dataset, test_loader = to_loader(test_inputs, test_labels, shuffle=True)

In [80]:
# Step 5: Fine-tune the pre-trained model
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

optimizer = AdamW(model.parameters(), lr=1e-5)

/opt/mamba/lib/python3.11/site-packages/transformers/optimization.py:457: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


In [ ]:
num_epochs = 3
for epoch in range(num_epochs):
    model.train()
    total_loss = 0
    for batch in tqdm(train_loader, desc="Epoch {}".format(epoch)):
        input_ids, attention_mask, labels = batch
        input_ids, attention_mask, labels = input_ids.to(device), attention_mask.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=labels)
        loss = outputs.loss
        total_loss += loss.item()
        loss.backward()
        optimizer.step()
    avg_train_loss = total_loss / len(train_loader)
    print("Average training loss:", avg_train_loss)

In [ ]:
# Step 6: Evaluate the fine-tuned model on the validation set
model.eval()
val_accuracy = 0
with torch.no_grad():
    for batch in tqdm(val_loader):
        input_ids, attention_mask, labels = batch
        input_ids, attention_mask, labels = input_ids.to(device), attention_mask.to(device), labels.to(device)
        outputs = model(input_ids=input_ids, attention_mask=attention_mask)
        logits = outputs.logits
        preds = torch.argmax(logits, dim=1)
        val_accuracy += torch.sum(preds == labels).item()

val_accuracy /= len(val_dataset)
print("Validation Accuracy:", val_accuracy)